In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from decision_tree import DT
from classsification_gd import ClassificationGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,auc,f1_score,precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets

import matplotlib.pyplot as plt




In [1]:
def update_class(X,main,other):
    result = []

    for i in range(len(X)):
        result.append(1 if X[i] == main else other)
        
    return np.array(result)

def to_multi_label(Y,clases):

    Y_result = []

    for class_ in Y:
        encode_for_elem = [0 for _ in range(len(clases.keys()))]
        encode_for_elem[clases[class_]] = 1
        Y_result.append(encode_for_elem)
    

    return np.array(Y_result)


class multi_svm:
    def __init__(self,alpha,epochs,c=10):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs
        self.c = c

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs,self.c,'svm'))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,-1)
            Y_test_ = update_class(Y_test,class_,-1)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].svm_raw_predict(point) for i in range(len(self.models))}
            # print(scores)
            
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 


In [3]:
class multi_logistic:
    def __init__(self,alpha,epochs):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,0)
            Y_test_ = update_class(Y_test,class_,0)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].logistic_h(point) for i in range(len(self.models))}
            
            # print(scores)
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 

In [ ]:
def k_folds(X,Y,k,model):
    pass

In [2]:

scaler = MinMaxScaler()




data = pd.read_csv('./proyect_dataset/training.csv')
data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
# data[data.columns[-1]] = update_class(data[data.columns[-1]],1,-1)
# data = data.to_numpy()

print(list({elem for elem in data[data.columns[-1]]}))
# print({elem for elem in update_class(data[data.columns[-1]],1,-1)})
# print(type(data['LB']))
# print(len(data['LB']))


data_train = data.sample(frac=0.7, random_state=43)
data_test = data[~data.index.isin(data_train.index)]




X = data.drop(columns='CLASE').to_numpy()
Y = data['CLASE'].to_numpy()


X_train = data_train.drop(columns='CLASE').to_numpy()
Y_train = data_train['CLASE'].to_numpy()

X_test = data_test.drop(columns='CLASE').to_numpy()
Y_test = data_test['CLASE'].to_numpy()




[1, 2, 3]


In [3]:

alpha = 0.15
epochs = 1000
c = 10
clases = list({elem for elem in data[data.columns[-1]]})

# model = multi_svm(alpha,epochs,c)
model = multi_logistic(alpha,epochs)
model.train(X_train,Y_train,X_test,Y_test,clases)


# model = DT(X_train,Y_train)







In [8]:

clases_map = {clases[i]:i for i in range(len(clases))}

predicted_data = model.predict(X_train)

# one_hot_data_pred = to_multi_label(predicted_data,clases_map):w

# one_hot_real_data = to_multi_label(Y_test,clases_map)

print(confusion_matrix(Y_train,predicted_data))
# print(precision_score(one_hot_real_data,one_hot_data_pred,average=None))
# print(recall_score(one_hot_real_data,one_hot_data_pred,average=None))
# print(f1_score(one_hot_real_data,one_hot_data_pred,average=None))
# 
# print(auc(Y_test,predicted_data))


[[1079    0    0]
 [   1  208    0]
 [   0    0  111]]
